In [2]:
import os
import json
import cv2
import albumentations as A
from pathlib import Path
from tqdm import tqdm
from copy import deepcopy

# 1. Data Augmentation

In [3]:


# Paths
dataset_path = Path("datasets/unified_dataset")
aug_dataset_path = Path("datasets/unified_dataset_data_augmentation")
aug_images_path = aug_dataset_path / "images"
aug_images_path.mkdir(parents=True, exist_ok=True)

# Load original annotations
with open(dataset_path / "_annotations.coco.json", 'r') as f:
    coco = json.load(f)

# Albumentations pipeline
transform = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.9),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
    A.MotionBlur(blur_limit=3, p=0.3)
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

# Prepare new data
new_images = []
new_annotations = []
new_image_id = max(img['id'] for img in coco['images']) + 1
new_annotation_id = max(ann['id'] for ann in coco['annotations']) + 1

image_id_to_anns = {}
for ann in coco['annotations']:
    image_id_to_anns.setdefault(ann['image_id'], []).append(ann)

# Process each image
for img_info in tqdm(coco['images'], desc="Augmenting images"):
    img_path = dataset_path / img_info['file_name']
    image = cv2.imread(str(img_path))
    if image is None:
        continue

    anns = image_id_to_anns.get(img_info['id'], [])
    bboxes = [ann['bbox'] for ann in anns]
    category_ids = [ann['category_id'] for ann in anns]

    if not bboxes:
        continue

    # Apply transformation
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

    # Save new image
    new_file_name = f"aug_{img_info['file_name']}"
    cv2.imwrite(str(aug_images_path / new_file_name), transformed['image'])

    # Add new image entry
    new_images.append({
        "id": new_image_id,
        "file_name": f"images/{new_file_name}",
        "width": img_info['width'],
        "height": img_info['height']
    })

    # Add new annotation entries
    for bbox, cat_id in zip(transformed['bboxes'], transformed['category_ids']):
        new_annotations.append({
            "id": new_annotation_id,
            "image_id": new_image_id,
            "category_id": cat_id,
            "bbox": bbox,
            "iscrowd": 0,
            "area": bbox[2] * bbox[3]
        })
        new_annotation_id += 1

    new_image_id += 1

# Create final annotation object
augmented_coco = deepcopy(coco)
augmented_coco['images'].extend(new_images)
augmented_coco['annotations'].extend(new_annotations)

# Save to new annotations file
with open(aug_dataset_path / "_annotations.coco.json", 'w') as f:
    json.dump(augmented_coco, f, indent=2)

print("Data augmentation complete and saved.")


C:\Users\Jose\AppData\Roaming\Python\Python312\site-packages\albumentations\core\validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
Augmenting images: 100%|██████████| 1238/1238 [02:22<00:00,  8.69it/s]


Data augmentation complete and saved.


# 1.1 Pruebas

In [37]:
import os
import json
import cv2
import albumentations as A
from pathlib import Path
from copy import deepcopy

# Paths
dataset_path = Path("datasets/unified_dataset")
aug_dataset_path = Path("datasets/unified_dataset_data_augmentation_v2")
aug_images_path = aug_dataset_path / "images"
aug_images_path.mkdir(parents=True, exist_ok=True)

# Load COCO annotations
with open(dataset_path / "_annotations.coco.json", 'r') as f:
    coco = json.load(f)

# Select a single image (first one)
img_info = coco['images'][0]
img_path = dataset_path / img_info['file_name']
image = cv2.imread(str(img_path))

# Get annotations for this image
image_id = img_info['id']
anns = [ann for ann in coco['annotations'] if ann['image_id'] == image_id]
bboxes = [ann['bbox'] for ann in anns]
category_ids = [ann['category_id'] for ann in anns]

# Albumentations transform (tweak here)
'''
transform = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.04, scale_limit=0.05, rotate_limit=10, p=1.0),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0),
    A.MotionBlur(blur_limit=3, p=1.0)
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))
'''
transform = A.Compose([
    A.ShiftScaleRotate(shift_limit=0, scale_limit=0, rotate_limit=10, p=1.0),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))
# Apply transformation
transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

# Save new image
new_file_name = f"aug_{img_info['file_name']}"
cv2.imwrite(str(aug_images_path / new_file_name), transformed['image'])

# Build new image + annotations
new_image_id = max(img['id'] for img in coco['images']) + 1
new_annotation_id = max(ann['id'] for ann in coco['annotations']) + 1

aug_img = {
    "id": new_image_id,
    "file_name": f"images/{new_file_name}",
    "width": img_info['width'],
    "height": img_info['height']
}

aug_anns = []
for bbox, cat_id in zip(transformed['bboxes'], transformed['category_ids']):
    aug_anns.append({
        "id": new_annotation_id,
        "image_id": new_image_id,
        "category_id": cat_id,
        "bbox": bbox,
        "iscrowd": 0,
        "area": bbox[2] * bbox[3]
    })
    new_annotation_id += 1

# Create new COCO structure with original + one augmented image
augmented_coco = deepcopy(coco)
augmented_coco['images'].append(aug_img)
augmented_coco['annotations'].extend(aug_anns)

# Save to new JSON
with open(aug_dataset_path / "_annotations.coco.json", 'w') as f:
    json.dump(augmented_coco, f, indent=2)

print(f"Augmented one image: {img_info['file_name']}")


Augmented one image: Foto-Placa-63-_jpg.rf.ffdc74fd31b2323a39646cfdfcecb8cc.jpg
